In [1]:
import yadage_widget
reload(yadage_widget)

<module 'yadage_widget' from 'yadage_widget.pyc'>

In [20]:
%%javascript
require.undef('yadage');
require.config({paths: {lukasvis: "http://cdnjs.cloudflare.com/ajax/libs/vis/4.17.0/vis"}});
define('yadage', ["jupyter-js-widgets","lukasvis"], function(widgets,vis) {

    var WorkflowWidgetView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed();
            this.model.on('change:dotstring', this.value_changed, this);
        },


        value_changed: function() {
            var data = this.model.get('dotstring');
            this.el.innerHTML = ''
            this.container = document.createElement('div');
            this.container.innerHTML = 'hello'
            this.el.appendChild(this.container);
            this.container.style.height="500px";
            this.container.style.width="500px";
            var DOTstring = data;
            console.log('hello')
            console.log(DOTstring)
            console.log('done')
            console.log(data)
            console.log('done2')
            var parsedData = vis.network.convertDot(DOTstring);
            this.netdata = {
              nodes: parsedData.nodes,
              edges: parsedData.edges
            }
            this.net_options = parsedData.options;
            this.network = new vis.Network(this.container, this.netdata, this.net_options);
            console.log(this.netdata.nodes)
        },
    });

    return {
        WorkflowWidgetView : WorkflowWidgetView
    };
});

<IPython.core.display.Javascript object>

In [21]:
w = yadage_widget.WorkflowWidget()
w

In [22]:
import yadage.helpers

In [23]:
import adage.visualize
import time
class ViewTracker(object):
    def __init__(self,widget):
        self.widget = widget
    def initialize(self,adageobj):
        pass
    def track(self,adageobj):
        self.widget.dotstring = adage.visualize.colorize_graph_at_time(adageobj.dag,time.time()).to_string() 
    def finalize(self,adageobj):
        self.widget.dotstring = adage.visualize.colorize_graph_at_time(adageobj.dag,time.time()).to_string() 

In [24]:
from yadage.yadagemodels import YadageWorkflow
from yadage.workflow_loader import workflow
from packtivity.statecontexts import poxisfs_context as statecontext
from adage import rundag
from yadage.clihelpers import setupbackend_fromstring
import shutil

workdir = 'fromipython'
try:
    shutil.rmtree(workdir)
except OSError:
    pass
wflowspec = workflow('madgraph_delphes.yml','from-github/phenochain')
rootcontext = statecontext.make_new_context(workdir)
wflow = YadageWorkflow.createFromJSON(wflowspec,rootcontext)

wflow.view().init({'nevents':100})
backend = setupbackend_fromstring('multiproc:4')
rundag(wflow, update_interval = 5, backend = backend, default_trackers=False, additional_trackers=[ViewTracker(w)])
